# Effect of possible errors in EM reconstruction 

From https://www.biorxiv.org/content/10.1101/2021.07.28.454025v3.full.pdf we have that 


```
We manually identified synapses in 70 small subvolumes (n=8,611 synapses) distributed across
the dataset, giving the automated detection an estimated precision of 96% and recall of 89%
(Supplemental Fig 1) with a partner assignment accuracy of 98%.
```

So we generate controls for our metrics were we: 

* Remove 4% of  the connections; taking into account the precision
* Add 12% ~ 1/0.89 of the connections; taking into account the recall
* Shuffle 2% of the connections; taking into account the partner accuracy

Note that this manipulations overestimate the possible errors, since they are done at the level of connections rather than synapses. 

In [1]:
import numpy as np
import pandas as pd
import scipy.sparse as sp

from connalysis import randomization 
from conntility import ConnectivityMatrix

import sys
sys.path.append('../../../library')
from structural_basic import load_connectome

In [2]:
# Load MICrONS connectome (central subvolume) 
data_dir="../../data"
conn=load_connectome(data_dir, 'MICrONS')
M=conn.matrix.astype('bool') # remove potential weights of synapses
no_edges=M.row.shape[0]

In [3]:
# Removing 4% of the edges at random
np.random.seed(10)
edges_remove=np.sort(np.random.choice(no_edges, size=int(no_edges*0.04), replace=False))
i=np.delete(M.row, edges_remove)
j=np.delete(M.col, edges_remove)
data=np.delete(M.data, edges_remove)
M_precision=sp.coo_matrix((data, (i, j)), shape=M.shape)
print(f"Is the new matrix a submatrix of the original? : {(M.multiply(M_precision)-M_precision).nnz==0}")
print(f"Using {M_precision.nnz*100/M.nnz}% of the edges")

Is the new matrix a submatrix of the original? : True
Using 96.0000153705159% of the edges


In [4]:
# Adding 12% of edges at random 
M_recall=randomization.add_connections(M, int(no_edges*0.12), seed=0)
print(f"Is the original matrix a submatrix of the new? : {(M.multiply(M_recall)-M).nnz==0}")
print(f"The original has {M.nnz*100/M_recall.nnz}% as many edges as the control")

/gpfs/bbp.cscs.ch/home/egassant/connalysis/lib/python3.10/site-packages/scipy/sparse/_index.py:143: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


Is the original matrix a submatrix of the new? : True
The original has 89.28575104557291% as many edges as the control


In [5]:
# Shuffle 2% of the edges at random 
no_edges_shuffle=int(no_edges*0.02)
# Remove 2% of the edges
edges_remove=np.sort(np.random.choice(no_edges, size=no_edges_shuffle, replace=False))
i=np.delete(M.row, edges_remove)
j=np.delete(M.col, edges_remove)
data=np.delete(M.data, edges_remove)
M_partner=sp.coo_matrix((data, (i, j)), shape=M.shape)
# Add edges back at random 
M_partner=randomization.add_connections(M_partner, no_edges_shuffle, seed=0)

print(f"The new matrix and the original agree in {(M.multiply(M_partner)).nnz*100/M.nnz}% of the entries")
print(f"They have the same number of edges? {M_partner.nnz==M.nnz}")


The new matrix and the original agree in 98.00029588243092% of the entries
They have the same number of edges? True


In [6]:
# Saving 
fn=f"{data_dir}/MICrONS_EM_error"
ConnectivityMatrix(M_precision, vertex_properties=conn.vertices).to_h5(f"{fn}_precision.h5")
ConnectivityMatrix(M_recall, vertex_properties=conn.vertices).to_h5(f"{fn}_recall.h5")
ConnectivityMatrix(M_partner, vertex_properties=conn.vertices).to_h5(f"{fn}_partner.h5")
